# Лечение гипертонии

72 пациента проходили лечение от гипертонии. Для лечения использовались три вида лекарств, при этом их эффект изучался как при использовании специальной диеты, так и в её отсутствии; кроме того, в половине случаев применялась психотерапия. Данные — артериальное давление пациента по окончании лечения.

Требуется сравнить эффективность методов лечения гипертонии.

## Загрузим данные и посмотрим на них

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.stats as st
import pandas as pd
import seaborn as sns

In [ ]:
data = pd.read_csv('hyper.txt', delimiter='\t')

In [ ]:
data.head()

In [ ]:
data.describe()

In [ ]:
data.sample(10)

In [ ]:
data['biofeedback'].describe()

In [ ]:
data['biofeedback'].unique()

### Трехфакторный дисперсионный анализ для пациентов

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [ ]:
lm = ols('pressure ~ C(biofeedback) * C(diet) * C(drug)', data=data).fit()
table = sm.stats.anova_lm(lm, typ=1)
table

Значимые взаимодействия

In [ ]:
table[table['PR(>F)'] < 0.05]

#### Построим графики взаимодействий

$$X_{ijk} = \mu + \alpha_i + \beta_j + \gamma_{ij} + \varepsilon_{ijk},$$
$$i = 1, ..., K_1,\ j = 1, ..., K_2,\ k = 1, ..., n.$$

- $\mu$ -- общее среднее значение признака,
- $\alpha_i$ -- воздействие уровня $i$ фактора $f_1$,
- $\beta_j$ -- воздействие уровня $j$ фактора $f_2$,
- $\gamma_{ij}$ -- дополнительное воздействие комбинации уровней $i$ и $j$ факторов $f_1$ и $f_2$,
- $\varepsilon_{ijk}$ -- случайные независимые одинаково распределенные ошибки

In [ ]:
plt.title('Diet: absent')
sns.pointplot(x="biofeedback", y="pressure", data=data.loc[data['diet']=='absent'], hue='drug')

plt.show()
plt.title('Diet: present')
sns.pointplot(x="biofeedback", y="pressure", data=data.loc[data['diet']=='present'], hue='drug')

In [ ]:
plt.title('biofeedback: absent')
sns.pointplot(x="diet", y="pressure", data=data.loc[data['biofeedback']=='absent'], hue='drug')

plt.show()
plt.title('biofeedback: present')
sns.pointplot(x="diet", y="pressure", data=data.loc[data['biofeedback']=='present'], hue='drug')

Тройное взаимодействие значимо, поэтому разделим выборку по одному из признаков.

### Двухфакторный дисперсионный анализ для пациентов, проходящих психотерапию

In [ ]:
lm = ols('pressure ~  C(diet) * C(drug)',data=data.loc[data['biofeedback']=='present']).fit()
table = sm.stats.anova_lm(lm)
table

Есть значимое межфакторное взаимодействие.

#### Графики

In [ ]:
_ = sns.pointplot(x="diet", y="pressure", data=data.loc[data['biofeedback']=='present'], hue='drug')
plt.show()
_ = sns.pointplot(x="drug", y="pressure", data=data.loc[data['biofeedback']=='present'], hue='diet')

Поэтому снова поделим выборку.

### Однофакторный анализ для пациентов, проходящих психотерапию и сидящих на диете:

In [ ]:
sns.boxplot(x='drug', y='pressure', data=data[(data['biofeedback']=='present') & (data['diet']=='present')])

In [ ]:
lm =  # ваш код
table = sm.stats.anova_lm(lm)
table

Ожидаемый p-value: ```~0.850126```

### Однофакторный анализ для пациентов, проходящих психотерапию и не сидящих на диете:

In [ ]:
_ = sns.boxplot(x='drug', y='pressure', data=data[(data['biofeedback']=='present') & (data['diet']=='absent')])

In [ ]:
lm = # ваш код
table = sm.stats.anova_lm(lm)
table

Ожидаемый p-value: ```~0.000269```

#### Исследуем различия в воздействии между группами

In [ ]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.multicomp import MultiComparison
# ваш код

print(result)

Препарат 1 значимо эффективнее двух других.

### Двухфакторный дисперсионный анализ для пациентов, не проходящих психотерапию:

In [ ]:
lm = ols('pressure ~ C(diet) * C(drug)',data=data[data['biofeedback']=='absent']).fit()
table = sm.stats.anova_lm(lm, typ=1)
table

Межфакторное взаимодействие незначимо:

In [ ]:
_ = sns.pointplot(x="diet", y="pressure", data=data[data['biofeedback']=='absent'], hue='drug')
plt.show()
_ = sns.pointplot(x="drug", y="pressure", data=data[data['biofeedback']=='absent'], hue='diet')

#### Исследуем попарные различия в воздействии каждого из лекарств в зависимости от диеты

In [ ]:
subset = # ваш код

In [ ]:
subset = # ваш код

## Итого:

* если пациент не проходит психотерапию, соблюдение диеты приводит к значимому уменьшению давления при приёме любого из трёх препаратов;
* эффективность всех трёх препаратов неразличима для пациентов, сидящих на диете и проходящих психотерапию;
* для пациентов, проходящих психотерапию и не сидящих на диете, препарат 1 эффективнее в среднем примерно на 20 мм. рт. ст.
* Чтобы сделать другие возможные выводы (например, оценить эффекты психотерапии и эффект диеты для тех, кто проходит психотерапию), нужно разбить трёхфакторный анализ по двум другим признакам.

**Maxwell S.E., Delaney H.D. Designing Experiments and Analyzing Data: A Model Comparison Perspective. (1990). Глава 8, стр. 325.**